<a href="https://colab.research.google.com/github/hakim3189/MachineLearning/blob/main/Pelatihan_Model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from google.colab import drive
!pip install gensim
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
!pip install gensim
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB # Import the BernoulliNB class
from sklearn.metrics import accuracy_score

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
main_df = pd.read_csv('/content/drive/MyDrive/ulasan_aplikasi_mytelkomsel.csv')

In [4]:
main_df.head()

,Review
0,Makin hari makin BURUK! segala upaya biar lanc...
1,Perbaiki dulu jaringannya paketnya sudah mahal...
2,Aku gak ngerti ya! Lama2 aplikasinya makin nge...
3,"Sebenarnya sudah mulai bosan, sebab masalahnya..."
4,"makin kesini aplikasi makin berat, buka aplika..."


Case Folding

In [5]:
main_df.loc[:, 'Review'] = main_df.loc[:, 'Review'].str.lower()
main_df['Review']

,Review
0,makin hari makin buruk! segala upaya biar lanc...
1,perbaiki dulu jaringannya paketnya sudah mahal...
2,aku gak ngerti ya! lama2 aplikasinya makin nge...
3,"sebenarnya sudah mulai bosan, sebab masalahnya..."
4,"makin kesini aplikasi makin berat, buka aplika..."
...,...
121495,"aplikasi trs yg di update bkn kualitas sinyal,..."
121496,"aplikasi indihome di alihkan ke sini ,tapi log..."
121497,"sekarang semakin gk bermutu nih aplikasi, pake..."
121498,soal sinyal memang lebih dari yg lain tapi har...


Removal Special Characters

In [6]:
!pip install Sastrawi
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
nltk.download('punkt_tab')

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy","gk"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "yg": "yang"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [8]:
nltk.download('stopwords') # This line downloads the 'stopwords' resource
nltk.download('punkt') # This line downloads the 'punkt' resource for tokenization
nltk.download('punkt_tab')

# Membersihkan teks dan menyimpannya di kolom 'text_clean'
main_df['text_clean'] = main_df['Review'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
main_df['text_casefoldingText'] = main_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
main_df['text_slangwords'] = main_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
main_df['text_tokenizingText'] = main_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
main_df['text_stopword'] = main_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
main_df['text_akhir'] = main_df['text_stopword'].apply(toSentence)

main_df['text_stopword']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,text_stopword
0,"[buruk, upaya, biar, lancar, masuk, aplikasi, ..."
1,"[perbaiki, jaringannya, paketnya, mahal, knapa..."
2,"[ngerti, aplikasinya, ngeleg, susah, update, k..."
3,"[bosan, seolaholah, diperbaiki, klaim, app, ke..."
4,"[kesini, aplikasi, berat, buka, aplikasi, cek,..."
...,...
121495,"[aplikasi, trs, update, bkn, kualitas, sinyal,..."
121496,"[aplikasi, indihome, alihkan, login, email, ap..."
121497,"[bermutu, nih, aplikasi, paket, mahal, isi, ri..."
121498,"[sinyal, harga, paket, dr, smoga, bsa, berubah]"


Labeling

In [19]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")


In [20]:
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
          score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
          score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 10):
      polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 10, maka polaritas adalah positif
    elif (score < 5):
      polarity = 'negative'
        # Jika skor sentimen kurang dari 5, maka polaritas adalah negatif
    else:
      polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [21]:
results = main_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
main_df['polarity_score'] = results[0]
main_df['polarity'] = results[1]
print(main_df['polarity'].value_counts())
main_df[['text_stopword','polarity','polarity_score']]

polarity
negative    103092
neutral      12265
positive      6143
Name: count, dtype: int64


,text_stopword,polarity,polarity_score
0,"[buruk, upaya, biar, lancar, masuk, aplikasi, ...",negative,-30
1,"[perbaiki, jaringannya, paketnya, mahal, knapa...",negative,-13
2,"[ngerti, aplikasinya, ngeleg, susah, update, k...",negative,1
3,"[bosan, seolaholah, diperbaiki, klaim, app, ke...",neutral,7
4,"[kesini, aplikasi, berat, buka, aplikasi, cek,...",negative,-25
...,...,...,...
121495,"[aplikasi, trs, update, bkn, kualitas, sinyal,...",negative,-5
121496,"[aplikasi, indihome, alihkan, login, email, ap...",negative,-8
121497,"[bermutu, nih, aplikasi, paket, mahal, isi, ri...",negative,-7
121498,"[sinyal, harga, paket, dr, smoga, bsa, berubah]",neutral,7


Ekstrasi Fitur

In [26]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = main_df['text_akhir']
y = main_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.6, random_state=60)

Pelatihan: naive_bayes,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 60/40

In [27]:
# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.8501440329218107
Naive Bayes - accuracy_test: 0.8501920438957477


Pelatihan: Logistic_Regression,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 60/40


In [28]:
# Membuat objek model Logistic_Regression
Logistic_Regression = LogisticRegression()

# Melatih model Logistic_Regression pada data pelatihan
Logistic_Regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = Logistic_Regression.predict(X_train.toarray())
y_pred_test_nb = Logistic_Regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic_Regression
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Logistic_Regression - accuracy_train:', accuracy_train_nb)
print('Logistic_Regression - accuracy_test:', accuracy_test_nb)

Logistic_Regression - accuracy_train: 0.8853497942386831
Logistic_Regression - accuracy_test: 0.8833744855967078


Pelatihan: RandomForest,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 60/40

In [29]:
# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9964814814814815
Random Forest - accuracy_test: 0.8655144032921811


In [30]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.8, random_state=20)

Pelatihan: RandomForest,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20

In [31]:
# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9976543209876543
Random Forest - accuracy_test: 0.8603395061728395


Pelatihan: Gradient Boosting Classifier,    Ekstraksi Fitur: TF-IDF,    Pembagian Data: 80/20

In [32]:
# Membuat objek model GradientBoosting
GBC = GradientBoostingClassifier()

# Melatih model GradientBoosting pada data pelatihan
GBC.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = GBC.predict(X_train.toarray())
y_pred_test_rf = GBC.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('GradientBoostingClassifier - accuracy_train:', accuracy_train_rf)
print('GradientBoostingClassifier - accuracy_test:', accuracy_test_rf)

GradientBoostingClassifier - accuracy_train: 0.8869135802469136
GradientBoostingClassifier - accuracy_test: 0.8642181069958847


In [33]:
 # Import train_test_split from sklearn.model_selection


y = main_df['polarity']

# 1. Prepare your data
sentences = main_df['text_stopword'].tolist()  # Assuming 'text_stopword' column contains preprocessed text

# 2. Train Word2Vec model (if you don't have a pre-trained one)
model = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
# Adjust parameters as needed

# 3. Function to extract features for a document
def document_vector(doc, model):
    """Calculates document vector by averaging word vectors."""
    doc_vec = np.zeros((model.vector_size,))
    word_count = 0
    for word in doc:
        if word in model.wv:
            doc_vec += model.wv[word]
            word_count += 1
    if word_count != 0:
        doc_vec /= word_count
    return doc_vec

# 4. Apply to your data
X_word2vec = [document_vector(doc, model) for doc in sentences]
X_word2vec = np.array(X_word2vec)

# Now you have X_word2vec as your feature matrix

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.8, random_state=42)


Pelatihan: Random Forest, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20

In [34]:
# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 1.0
Random Forest - accuracy_test: 0.8567592592592592


Pelatihan: MLP Classifier, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20

In [35]:
# Membuat objek model MLP_Classifier
MLP_Classifier = MLPClassifier()

# Melatih model MLP_Classifier pada data pelatihan
MLP_Classifier.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = MLP_Classifier.predict(X_train)
y_pred_test_rf = MLP_Classifier.predict(X_test)

# Evaluasi akurasi model MLP_Classifier
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('MLP_Classifier - accuracy_train:', accuracy_train_rf)
print('MLP_Classifier - accuracy_test:', accuracy_test_rf)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP_Classifier - accuracy_train: 0.9555144032921811
MLP_Classifier - accuracy_test: 0.8500823045267489


Pelatihan: Linear SVC, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20

In [36]:
# Membuat objek model Linear_SVC
Linear_SVC = LinearSVC()

# Melatih model Linear_SVC pada data pelatihan
Linear_SVC.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = Linear_SVC.predict(X_train)
y_pred_test_rf = Linear_SVC.predict(X_test)

# Evaluasi akurasi model Linear_SVC
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Linear_SVC - accuracy_train:', accuracy_train_rf)
print('Linear_SVC - accuracy_test:', accuracy_test_rf)

Linear_SVC - accuracy_train: 0.8579835390946502
Linear_SVC - accuracy_test: 0.8556172839506173


In [37]:
pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.2.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.11
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
ale-py==0.10.1
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==6.1.7
astropy-iers-data==0.2025.1.27.0.32.44
astunparse==1.6.3
atpublic==4.1.0
attrs==25.1.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.34.0
bigquery-magics==0.5.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.0.0
bokeh==3.6.2
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.1
catalogue==2.0.10
certifi==2024.12.14
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.88
clarabel==0.9.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.1
cmake==3.31.4
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contourpy==1.3.1
